In [1]:
# Params
input_size = (299,299)
batch_size = 100
num_workers = 8
num_classes = 2

In [2]:
import torch

print("torch-version:", torch.__version__)
print("Available GPU:", torch.cuda.current_device())

torch-version: 1.1.0
Available GPU: 0


In [3]:
from load_MH import *

# 기존 데이터 경로
NORMAL_DIR = '../../snapshots_pureok'
FAULT_DIR = '../../snapshots'
# 추가 데이터 경로
ADD_NORMAL_DIR = '../../snapshots_punch181'
ADD_FAULT_DIR = '../../snapshots_punch181_NG'

train_mhdb = dataset(NORMAL_DIR, FAULT_DIR, ADD_NORMAL_DIR, ADD_FAULT_DIR, input_size, batch_size, num_workers)[:4]
val_mhdb = dataset(NORMAL_DIR, FAULT_DIR, ADD_NORMAL_DIR, ADD_FAULT_DIR, input_size, batch_size, num_workers)[4:8]
test_mhdb = dataset(NORMAL_DIR, FAULT_DIR, ADD_NORMAL_DIR, ADD_FAULT_DIR, input_size, batch_size, num_workers)[8:]

In [4]:
import torchvision

correct = 0
total = 0

CAM1 = "./saved_models/_CAM1-Epoch-88-Loss-1.978811.pth"
CAM2 = None
CAM3 = None
CAM4 = None

if CAM1:
    test_mhdb_batch = test_mhdb[0]
    CAM = CAM1
elif CAM2:
    test_mhdb_batch = test_mhdb[1]
    CAM = CAM2
elif CAM3:
    test_mhdb_batch = test_mhdb[2]
    CAM = CAM3
elif CAM4:
    test_mhdb_batch = test_mhdb[3]
    CAM = CAM4
    
net = torchvision.models.inception_v3(pretrained=False, aux_logits=True, num_classes=num_classes).cuda()
net.load_state_dict(torch.load(CAM))

with torch.no_grad():
    net.eval()
    for data in test_mhdb_batch:
        images, labels = data
        images = images.cuda()
        labels = labels.cuda().long()
        
        outputs = net(images)
                
        _, predicted = torch.max(outputs, dim=1)
        
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

print('Accuracy of the network on the test images: %d %%' %(100 * correct / total)) #more than 50 percent

Accuracy of the network on the test images: 89 %
